In [1]:
import os
import sys
import csv
import xlsxwriter

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\tubxt2p\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# User input

base = "tblTopsMatch.csv"
match = "tblWegmansMatch.csv"
base_field = "topsMatch"
match_field = "wegmansMatch"
method = "sort"
threshold = 60

In [3]:
def main():
    try:
        data_dir = ".\\excel\\data\\"
        archive_dir = ".\\excel\\archive\\"

        # Open base file 
        with open(data_dir + base, "r") as file:
            base_file = csv.DictReader(file)
            base_lst = []
            header_lst = []
            
            # Copy dictionary rows into a list, and extract headers into a list
            for row in base_file:
                for key in row:
                    if key not in header_lst:
                        header_lst.append(key)
                base_lst.append(row)
        
        # Open file to match records against the base file
        with open(data_dir + match, "r") as file:
            match_file = csv.DictReader(file)
            match_lst = []
            
            # Copy dictionary rows into a list
            for row in match_file:
                match_lst.append(row)
        
        # For dictionary row in the base file list...
        write_lst = []
        for base_row in base_lst:
            best_match = ("No Match", 0)
            row_lst = []         
            # Match each row against the current base row
            for match_row in match_lst:
                match_ratio = fuzzy_match(base_row[base_field], match_row[match_field], method)                
                # If the match ratio is less than threshold, skip record
                if match_ratio < threshold:
                    continue
                # Else, assign the highest ratio (linear search)
                elif match_ratio > best_match[1]:
                    best_match = (match_row[match_field], match_ratio)
            print(f"[{base_row[base_field]} | {best_match[0]}] Match Ratio: {best_match[1]}")
            
            # For each row bring in all additional fields from base file, using header as key
            for header in header_lst:
                row_lst.append(base_row[header])    
                
            # For each row, create a list of values to be appeneded to a master write list
            row_lst.extend(list(best_match))
            write_lst.append(row_lst)
        
        # Create a new workbook and worksheet
        workbook = xlsxwriter.Workbook(archive_dir + f"match-{method}.xlsx")
        worksheet = workbook.add_worksheet()
        # Write coloumn headers as the first row
        for col_num, data in enumerate(header_lst):
            worksheet.write(0, col_num, data)
        # Write each row from the write list to workbook
        for row_num, row_data in enumerate(write_lst):
            if row_num == 0:
                continue
            for col_num, col_data in enumerate(row_data):
                worksheet.write(row_num, col_num, col_data)
        workbook.close()
    except:
        if not os.path.exists(archive_dir):
            os.makedirs(archive_dir)
        if not os.path.exists(data_dir):
            os.makedirs(data_dir)

In [4]:
def fuzzy_match(base, match, method):
    """
    
    """
    if method == "ratio":
        return fuzz.ratio(base.lower(), match.lower())
    elif method == "pratio":
        return fuzz.partial_ratio(base.lower(), match.lower())
    elif method == "sort":
        return fuzz.token_sort_ratio(base, match)
    elif method == "set":
        return fuzz.token_set_ratio(base, match)
    else:
        print("ERROR: Invalid match method.")
        raise

In [5]:
main()

[MAYONNAISE REAL 30 | MAYONNAISE CLASSIC 30] Match Ratio: 67
[MAYONNAISE REAL 15 | MAYONNAISE CLASSIC 15] Match Ratio: 67
[VINEGAR APPLE CIDER 16 | VINEGAR CIDER 16] Match Ratio: 84
[VINEGAR CIDER 32 | VINEGAR CIDER 32] Match Ratio: 100
[ORGANIC VINEGAR APPLE CIDER 32 | ORGANIC VINEGAR APPLE CIDER 33.8] Match Ratio: 94
[VINEGAR WHITE 16 | VINEGAR WHITE 128] Match Ratio: 91
[VINEGAR WHITE 32 | VINEGAR WHITE 128] Match Ratio: 91
[VINEGAR WHITE 128 | VINEGAR WHITE 128] Match Ratio: 100
[ORGANIC KETCHUP SQUEEZE 20 | ORGANIC KETCHUP 20] Match Ratio: 82
[ORGANIC RELISH SWEET 10 | ORGANIC RELISH SWEET 8] Match Ratio: 93
[STEAK SAUCE 10 | STEAK SAUCE 10] Match Ratio: 100
[ORGANIC JUICE APPLE 100% 64 | ORGANIC JUICE 100% GRAPE WHITE 64] Match Ratio: 83
[ORGANIC 100% JUICE CRANBERRY BLEND 64 | ORGANIC JUICE 100% CRANBERRY 64] Match Ratio: 91
[ORGANIC JUICE GRAPE 100% 64 | ORGANIC JUICE 100% GRAPE WHITE 64] Match Ratio: 90
[ORGANIC 100% JUICE WHITE GRAPE 64 | ORGANIC JUICE 100% GRAPE WHITE 64] Ma

[FROZEN FRIES CRINKLE CUT 80 | FROZEN POTATOES CRINKLE CUT 80] Match Ratio: 74
[CUBE SIZE ICE NUGGETS 7 | No Match] Match Ratio: 0
[ORGANIC MILK 1% LOW FAT 64 | ORGANIC MILK 1% LF HG 64] Match Ratio: 83
[ICE THE BIG BAG 20 | ICE BAG 20] Match Ratio: 71
[ORGANIC MILK FAT FREE/SKIM 64 | MILK FAT FREE/SKIM NATURAL 64] Match Ratio: 83
[ICE THE BIG BAG 20 | ICE BAG 20] Match Ratio: 71
[LEMONADE 15% REFRIGERATED 52 | LEMONADE COLD PRESSED REFRIGERATED 59] Match Ratio: 75
[CRESCENT ROLLS REGULAR RFG 8CT 8 | CRESCENT ROLLS RFG 8] Match Ratio: 77
[PIE CRUST ROLLED RFG 15 | PIE CRUST DOUGH ROLLED 9IN RFG 2CT 15] Match Ratio: 77
[COLBY JACK SLC CRACKER CUTS 10 | COLBY JACK CUTS 10] Match Ratio: 75
[CREAM CHEESE SOFT FF 8 | CREAM CHEESE SOFT LIGHT 8] Match Ratio: 85
[PEPPER JACK SLC CRACKER CUTS 10 | PEPPER JACK CUTS 10] Match Ratio: 76
[SWISS SLC DELI 8 | CHEESE SWISS SLICED DELI 8] Match Ratio: 76
[SWISS SLICE AGED DELI 8 | CHEESE SWISS SLICED DELI 8] Match Ratio: 82
[SWISS SLC DELI THIN/SHINGLE

[BABY SHAMPOO LAVENDER/CHAMOMILE 15 | BABY SHAMPOO LAVENDER/CHAMOMILE 15] Match Ratio: 100
[AMMONIA LEMON SCENT 64 | AMMONIA LEMON 64] Match Ratio: 84
[SODA BTLS LEMON LIME 2 LITER 67.6 | SODA BTLS W-UP LEMON LIME 2 LITER 67.6] Match Ratio: 93
[SODA CANS LEMON LIME 12 | SODA CANS W-UP LEMON LIME 12] Match Ratio: 90
[PEDIATRIC ELECTROLYTE SOLUTION ADVANTAGE CARE STRAWBERRY LEMONADE 33.8 | PEDIATRIC ELECTROLYTE SOLUTION ADVANTAGE CARE STRAWBERRY LEMONADE 33.8] Match Ratio: 100
[ORGANIC BROWN SUGAR LIGHT 1.5 | ORGANIC SUGAR LIGHT BROWN PURE CANE 1.5] Match Ratio: 85
[PANCAKE SYRUP LT 24 | PANCAKE/WAFFLE SYRUP LT 24] Match Ratio: 84
[COFFEE GRND COLOMBIAN 12 | COFFEE GROUND COLOMBIAN 10.5] Match Ratio: 88
[COFFEE GRND COLOMBIAN 11 | COFFEE GROUND COLOMBIAN 10.5] Match Ratio: 88
[COFFEE GRND BREAKFAST BLEND 12 | COFFEE GRND BREAKFAST BLEND LT RST 13] Match Ratio: 87
[COFFEE BREAKFAST BLEND 11 | COFFEE WB BREAKFAST BLEND LT RST 13] Match Ratio: 80
[COFFEE GRND DECAF BREAKFAST BLEND 12 | COFF

[DRESSING FRENCH 16 | DRESSING FRENCH 16] Match Ratio: 100
[DRESSING HONEY MUSTARD 16 | DRESSING AVOCADO OIL HONEY MUSTARD 10] Match Ratio: 77
[DRESSING ITALIAN ROBUSTO 16 | DRESSING ITALIAN PARMESAN 16] Match Ratio: 80
[DRESSING RANCH BACON 16 | DRESSING RANCH CRMY 16] Match Ratio: 84
[CAT FOOD CAN BEEF TENDER 3 | CAT FOOD CAN TENDER BEEF DELT 3] Match Ratio: 91
[CAT FOOD CAN GOURMET CHICKEN GRILLED 3 | CAT FOOD CAN CHICKEN 3] Match Ratio: 73
[CAT FOOD CAN SALMON SAVORY 3 | CAT FOOD CAN SAVORY SALMON DELT 3] Match Ratio: 92
[CAT FOOD CAN TUNA GRILLED/GRAVY 3 | CAT FOOD CAN TURKEY SLICED GRAVY 3] Match Ratio: 81
[GOUDA SLICE DELI 8 | MOZZARELLA SLC DELI 8] Match Ratio: 67
[SODA BTLS BLUE RASPBERRY 2 LITER 67.6 | SODA BTLS ROOT BEER 2 LITER 67.6] Match Ratio: 78
[SODA BTLS CHERRY 2 LITER 67.6 | SODA BTLS BLK CHERRY 2 LITER 67.6] Match Ratio: 94
[SODA BTLS DIET ORANGE 2 LITER 67.6 | SODA BTLS ORNG 2 LITER 67.6] Match Ratio: 89
[SODA BTLS GRAPE 2 LITER 67.6 | SODA BTLS COLA 2 LITER 67.6] 

[ICE CRM CHOC CHIP 48 | ICE CREAM MINT CHOC CHIP 16] Match Ratio: 77
[ICE CRM CHOC CHIP DOUGH 48 | ICE CREAM MINT CHOC CHIP 16] Match Ratio: 68
[ICE CRM CHOC MARSHMALLOW SWIRL 48 | ICE CREAM CHOCOLATE MARSHMALLOW NATURAL 48] Match Ratio: 80
[ICE CRM CHOC PEANUT BUTTER SWIRL 48 | ICE CREAM LIGHT PEANUT BUTTER SWIRL NATURAL 48] Match Ratio: 74
[ICE CRM VANILLA CHOC 48 | ICE CREAM CHOCOLATE VANILLA NATURAL 48] Match Ratio: 75
[ICE CREAM COCONUT DREAM 48 | ICE CREAM COCONUT ALMOND FUDGE 48] Match Ratio: 78
[ICE CREAM CONFETTI CAKE 48 | ICE CREAM LIGHT COOKIES & CREAM 48] Match Ratio: 66
[ICE CRM COOKIES/CREAM 48 | ICE CREAM LIGHT COOKIES & CREAM 48] Match Ratio: 86
[ICE CRM DEATH BY CHOC 48 | ICE CREAM CHOCOLATE ALMOND 48] Match Ratio: 64
[ICE CRM CARAMEL PEANUT BUTTER BROWNIE 48 | ICE CREAM PEANUT BUTTER CUP 16] Match Ratio: 71
[ICE CREAM BLUEBERRY MUFFIN 48 | ICE CREAM BUTTER PECAN 16] Match Ratio: 63
[ICE CRM FRENCH VANILLA 48 | ICE CREAM FRENCH VANILLA NATURAL 48] Match Ratio: 83
[ICE 

...

[CEREAL CHEX EMULATION CORN 12 | CEREAL CHEX EMULATION RICE NATURAL 12] Match Ratio: 73
[CEREAL CORN FLAKES 18 | CEREAL CORN FLAKES NATURAL 18] Match Ratio: 84
[CEREAL CHEX EMULATION RICE 12 | CEREAL CHEX EMULATION RICE NATURAL 12] Match Ratio: 88
[CEREAL CINNAMON TOAST CRUNCH EMULATION 12.2 | CEREAL CINNAMON TOAST CRUNCH EMULATION NATURAL 12] Match Ratio: 89
[CEREAL BRAN FLAKES ENRICHED 17.3 | CEREAL FIBER BRAN FLAKES NATURAL 17.3] Match Ratio: 75
[CEREAL OATS/MORE HONEY ROASTED 14.5 | PEANUTS DRY ROASTED HONEY 16] Match Ratio: 70
[CEREAL OATS/MORE ALMOND 14.5 | ALMONDS ROASTED NATURAL 10.25] Match Ratio: 67
[CEREAL FRUIT RINGS 12.2 | CEREAL CORN SQUARES 12] Match Ratio: 62
[CEREAL FROSTED FLAKES 15 | CEREAL FROSTED FLAKES NATURAL 17] Match Ratio: 82
[OATS OLD FASHIONED 18 | OATS - OLD FASHIONED OR QUICK QUICK NATURAL 18] Match Ratio: 65
[OATS QUICK 18 | ICE BAG 18] Match Ratio: 61
[OATS OLD FASHIONED 42 | OATS - OLD FASHIONED OR QUICK QUICK NATURAL 42] Match Ratio: 65
[OATS QUICK 42 

[FROZEN ONIONS CHOPPED 12 | FROZEN ONIONS CHOPPED DICED NATURAL 10] Match Ratio: 74
[FROZEN PEAS PETITE 16 | FROZEN PEAS PETITE NATURAL 16] Match Ratio: 84
[FROZEN PEAS GREEN 16 | FROZEN PEAS SWEET NATURAL 16] Match Ratio: 67
[FROZEN PEAS CARROTS 16 | FROZEN PEAS CARROTS NATURAL 16] Match Ratio: 85
[FROZEN SPINACH CHOPPED 10 | FROZEN SPINACH CHOPPED NATURAL 12] Match Ratio: 83
[FROZEN SPINACH CUT LEAF 16 | FROZEN SPINACH CUT LEAF NATURAL 16] Match Ratio: 87
[FROZEN BEANS LIMA BABY 16 | FROZEN BEANS LIMA BABY NATURAL 16] Match Ratio: 86
[FROZEN MIXED VEGETABLES 16 | FROZEN MIXED VEGETABLES NATURAL 16] Match Ratio: 87
[FROZEN GREEN BEANS FRENCH SLC 16 | FROZEN GREEN BEANS FRENCH SLICE NATURAL 16] Match Ratio: 86
[FROZEN GREEN BEANS ITALIAN CUT 16 | FROZEN GREEN BEANS ITALIAN CUT NATURAL 16] Match Ratio: 89
[FROZEN GREEN BEANS WHL 16 | FROZEN GREEN BEANS WHOLE NATURAL 16] Match Ratio: 83
[FROZEN GREEN BEANS CUT 16 | FROZEN GREEN BEANS CUT NATURAL 16] Match Ratio: 86
[FROZEN BRUSSELS SPROU

[SPREAD BUTTER SALTED 15 | BUTTER WHIPPED SALTED 8] Match Ratio: 78
[ORANGE JUICE NOT FROM CONCENTRATE 100% 52 | ORGANIC JUICE 100% APPLE FROM CONCENTRATE 64] Match Ratio: 77
[ORANGE JUICE PREMIUM CARAFE 100% REFRIGERATED 52 | No Match] Match Ratio: 0
[ORANGE JUICE NOT FROM CONCENTRATE 100% NO PULP 52 | FROZEN CONC ORANGE JUICE 100% NO PULP NATURAL 12] Match Ratio: 78
[FROZEN STEAM RICED CAULIFLOWER 10.5 | FROZEN CAULIFLOWER RICED ASIAN STYLE 12] Match Ratio: 84
[BBQ SAUCE REGULAR 18 | BLEACH REGULAR SCENT 121] Match Ratio: 68
[BBQ SAUCE HICKORY 18 | ORGANIC SAUCE BBQ ORIGINAL 18] Match Ratio: 69
[BBQ SAUCE HONEY 18 | BBQ SAUCE BOURBON NATURAL 18] Match Ratio: 65
[POTATO SKINS CHEDDAR BACON 7.6 | FROZEN POTATO SKINS BACON CHEDDAR NATURAL 7.6] Match Ratio: 80
[TEA BAGS EARL GREY 20 | TEA BAGS GREEN 40] Match Ratio: 74
[TEA BAGS ENGLISH BREAKFAST 20 | TEA BAGS FLAVORS ENGLISH BREAKFAST NATURAL 20] Match Ratio: 78
[YOGURT LIGHT 6 | YOGURT LIGHT RASPBRY 6] Match Ratio: 78
[YOGURT LIGHT APP

[ORGANIC COFFEE K-CUPS FRENCH ROAST 10 | COFFEE K-CUPS FRENCH VANILLA NATURAL 12] Match Ratio: 74
[ORGANIC COFFEE K-CUPS COLUMBIAN 10 | COFFEE K-CUPS COLOMBIAN NATURAL 12] Match Ratio: 79
[ORGANIC COFFEE K-CUPS BREAKFAST BLEND 10 | COFFEE K-CUPS BREAKFAST BLEND NATURAL 12] Match Ratio: 85
[FRUIT CUPS MANDARIN ORANGE 4 | FRUIT CUPS PINEAPPLE NATURAL 4] Match Ratio: 66
[FRUIT BOWLS MIXED FRUIT 4 | FRUIT CUPS MIXED FRUIT NATURAL 4] Match Ratio: 74
[FRUIT CUPS MIXED FRUIT CHERRY 4 | FRUIT CUPS MIXED FRUIT CHERRY NATURAL 4] Match Ratio: 89
[FRUIT BOWLS PEACH DICED 4 | ORGANIC PEACHES/PEARS DICED CUPS 4] Match Ratio: 61
[FRUIT BOWLS PEAR DICED 4 | No Match] Match Ratio: 0
[FRUIT CUPS PINEAPPLE TIDBITS 4 | FRUIT CUPS PINEAPPLE NATURAL 4] Match Ratio: 73
[FRUIT CUPS TROPICAL FRUIT MIX 4 | FRUIT CUPS MIXED FRUIT NATURAL 4] Match Ratio: 86
[PASTA SAUCE TOMATO/GARLIC/ONION 24 | PASTA SAUCE HERB TOMATO BASIL NATURAL 24] Match Ratio: 68
[TORTILLA CHIPS NACHO 9 | CHIPS TORTILLA NACHO CHEESE FAMILY P

[ORGANIC OLIVE OIL EXTRA VIRGIN MARASCA 33.8 | ORGANIC OLIVE OIL EXTRA VIRGIN 33.8] Match Ratio: 90
[EURO SODA BLOOD ORANGE 33.8 | SODA EUROPEAN BLOOD ORANGE 33.8] Match Ratio: 93
[MOZZARELLA BALL WHL MILK 16 | SHRED MOZZARELLA WHOLE MILK NATURAL 16] Match Ratio: 68
[PESTO SAUCE/SPREAD 8 | No Match] Match Ratio: 0
[FROZEN PIZZA MARGHERITA 15.8 | FRZ PIZZA LIGURIA 14.32] Match Ratio: 67
[FROZEN PIZZA QUATRO FRM 16.2 | FRZ PIZZA PUGLIA 13.61] Match Ratio: 60
[FROZEN PIZZA SPINACH GOAT CHEESE 16.3 | FROZEN SPINACH CHOPPED NATURAL 12] Match Ratio: 69
[FROZEN PIZZA ROASTED VEGETABLES 17.3 | FROZEN STEAM MIXED VEGETABLES NATURAL 12] Match Ratio: 68
[EURO SODA LEMON & MANGO 33.8 | SODA EUROPEAN SICILIAN LEMON 33.8] Match Ratio: 78
[EURO SODA ITALIAN LEMON 33.8 | SODA EUROPEAN SICILIAN LEMON 33.8] Match Ratio: 72
[EURO SODA POMEGRANATE 33.8 | SODA EUROPEAN BLOOD ORANGE 33.8] Match Ratio: 70
[PASTA SAUCE VODKA 23.5 | PASTA SAUCE VODKA BLUSH 23.5] Match Ratio: 88
[ORGANIC VINEGAR BALSAMIC 17 | O